# BLEND027
- July 24
- Automate the Blend Weight Selection
- Only Use 3 Folds

In [1]:
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

def log_mean_absolute_error(y_true, y_pred):
    return np.log(mean_absolute_error(y_true, y_pred))
%matplotlib inline

In [2]:
types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC','3JHH','3JHC', '3JHN']

def get_sub_oof(t):
    """
    Grab the sub and oof files for a give type. 
    """
    oof = []
    sub = []
    model_ids = []
    for file in sorted(os.listdir(f'../type_results/{t}/')):
        if ('sub' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            model_ids.append(model_id)
            df = pd.read_parquet(f'../type_results/{t}/{file}')
            if len(sub) == 0:
                sub = df.rename(columns={'scalar_coupling_constant': model_id})
            else:
                sub[model_id] = df['scalar_coupling_constant']
        elif ('oof' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            #print(model_id)
            df = pd.read_parquet(f'../type_results/{t}/{file}')
            if len(oof) == 0:
                oof = df.rename(columns={'oof_preds': model_id})
            else:
                oof[model_id] = df['oof_preds']
    sub = sub.loc[sub['type'] == t]
    oof = oof.loc[oof['type'] == t]
    return sub, oof, model_ids

def get_best_weights(oof, l5, t):
    if len(l5) == 1:
        return [1]
    lmae_best = 0
    for x in tqdm(range(0, 5000)):
        ws = []
        for x in range(0, len(l5)):
            if x == len(l5)-1:
                ws.append(random.randint(1, 50))
            else:
                ws.append(random.randint(0, 50))
        idx = 0
        blend = np.zeros(len(oof[l5[0]]))
        for w in ws:
            blend += w * oof[l5[idx]]
            idx += 1
        blend = blend / np.sum(ws)
        lmae = log_mean_absolute_error(oof['scalar_coupling_constant'],
                            blend)
        if lmae < lmae_best:
            lmae_best = lmae
            bws = ws
    print(f'{t}: Best score {lmae:0.5f} - with weights {bws}')
    return bws # Return the best weights

def create_best_blends():
    oofs = []
    subs = []
    for t in types:
        print(f'{t}: Running for type {t}')
        tsub, toof, tmodel_ids = get_sub_oof(t)
        print(f'{t}: Has model names {tmodel_ids}')
        l5 = tmodel_ids[-5:] # Last 5 models
        bws = get_best_weights(toof, l5, t)
        tot_weight = np.sum(bws)
        tsub['scalar_coupling_constant'] = 0
        toof['oof_blend'] = 0
        idx = 0
        for w in bws:
            tsub['scalar_coupling_constant'] += (w * tsub[l5[idx]])
            toof['oof_blend'] += (w * toof[l5[idx]])
            idx += 1
        tsub['scalar_coupling_constant'] /= tot_weight
        toof['oof_blend'] /= tot_weight
        tsub = tsub[['id','molecule_name','type','scalar_coupling_constant']].copy()
        toof = toof[['id','type','scalar_coupling_constant','oof_blend']].copy()
        subs.append(tsub)
        oofs.append(toof)
    final_oof = pd.concat(oofs).sort_values('id').reset_index(drop=True)
    final_sub = pd.concat(subs).sort_values('id').reset_index(drop=True)
    return final_oof, final_sub

final_oof, final_sub = create_best_blends()
print('Done')

1JHC: Running for type 1JHC


  0%|          | 3/5000 [00:00<03:15, 25.51it/s]

1JHC: Has model names ['M047-0.7977', 'M048-0.8258', 'M049-0.791']


100%|██████████| 5000/5000 [02:41<00:00, 31.07it/s]


1JHC: Best score -0.94161 - with weights [24, 27, 25]
2JHH: Running for type 2JHH


...

3JHC: Best score -1.68522 - with weights [48, 47]
3JHN: Running for type 3JHN


  0%|          | 7/5000 [00:00<01:23, 60.11it/s]

3JHN: Has model names ['M047-2.356', 'M048-2.321', 'M049-2.156', 'M050-2.383', 'M052-2.39']


100%|██████████| 5000/5000 [01:23<00:00, 62.08it/s]


3JHN: Best score -2.48445 - with weights [24, 14, 3, 37, 16]
Done


# Score

In [3]:
for i, d in final_oof.groupby('type'):
    score = mean_absolute_error(d['scalar_coupling_constant'], d['oof_blend'])
    lscore = np.log(mean_absolute_error(d['scalar_coupling_constant'], d['oof_blend']))
    print(i,'\t {:0.5f} \t {:0.5f}'.format(score, lscore))
glmae = group_mean_log_mae(final_oof['scalar_coupling_constant'], final_oof['oof_blend'], final_oof['type'])
print('\nGroup LMAE')
print('{:0.5f}'.format(glmae))

1JHC 	 0.38889 	 -0.94447
1JHN 	 0.23413 	 -1.45186
2JHC 	 0.15514 	 -1.86340
2JHH 	 0.08663 	 -2.44613
2JHN 	 0.10113 	 -2.29140
3JHC 	 0.18144 	 -1.70684
3JHH 	 0.10256 	 -2.27729
3JHN 	 0.08079 	 -2.51591

Group LMAE
-1.93716


# Validate Sub

In [4]:
sub_good = pd.read_csv('../submissions/BLEND021_sub_-1.92084CV.csv')
final_sub['good_scc'] = sub_good['scalar_coupling_constant']
final_sub[['scalar_coupling_constant','good_scc']].corr()

,scalar_coupling_constant,good_scc
scalar_coupling_constant,1.000000,0.999998
good_scc,0.999998,1.000000


In [5]:
print('MAE vs Good Sub')
for i, d in final_sub.groupby('type'):
    mae_t = mean_absolute_error(d['good_scc'], d['scalar_coupling_constant'])
    corr_t = np.corrcoef(d['good_scc'], d['scalar_coupling_constant'])[1][0]
    print(f'{i} - {mae_t:0.5f} - {corr_t:0.5f}')

MAE vs Good Sub
1JHC - 0.00078 - 1.00000
1JHN - 0.04558 - 0.99998
2JHC - 0.00033 - 1.00000
2JHH - 0.00444 - 1.00000
2JHN - 0.03168 - 0.99989
3JHC - 0.06022 - 0.99943
3JHH - 0.02256 - 0.99995
3JHN - 0.00600 - 0.99997


# Create Submission and OOF Files

In [6]:
import json
import os.path
import re
import ipykernel
import requests

#try:  # Python 3
#    from urllib.parse import urljoin
#except ImportError:  # Python 2
#    from urlparse import urljoin

# Alternative that works for both Python 2 and 3:
from requests.compat import urljoin

try:  # Python 3 (see Edit2 below for why this may not work in Python 2)
    from notebook.notebookapp import list_running_servers
except ImportError:  # Python 2
    import warnings
    from IPython.utils.shimmodule import ShimWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ShimWarning)
        from IPython.html.notebookapp import list_running_servers


def get_notebook_name():
    """
    Return the full path of the jupyter notebook.
    """
    kernel_id = re.search('kernel-(.*).json',
                          ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'),
                                params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

In [7]:
BLEND_NUMBER = get_notebook_name().split('/')[-1].replace('.ipynb','').replace('-','')
print(f'NAME TO SAVE {BLEND_NUMBER}')

NAME TO SAVE BLEND027


In [8]:
# Save the Results
final_sub[['id','scalar_coupling_constant']].to_csv(f'../submissions/{BLEND_NUMBER}_sub_{glmae:0.5f}CV.csv', index=False)
final_oof.to_csv(f'../oof/{BLEND_NUMBER}_oof_{glmae:0.5f}CV.csv', index=False)